In [13]:
from utils import AnalyzeVolume
import os
import pdb
import SimpleITK as sitk

root_folder_pcct = r'E:\closed_elastix_cartilages2'
root_folder_pcct_images = os.listdir(root_folder_pcct)

# for image in root_folder_pcct_images:
#     if os.path.splitext(image)[1] == '.nii':
#         knee = image.split('_')[0] + '_' + image.split('_')[1]
gt = os.path.join(r'E:\segmentations MR Tim\07_2017\07_2017_tibial_cartilage.nrrd')
pred = os.path.join(root_folder_pcct,'07_2017_tibial_cartilage.nii' )
        
gt_itk = sitk.ReadImage(gt)
        
pred_itk = sitk.ReadImage(pred)


resample = sitk.ResampleImageFilter()
resample.SetOutputSpacing(gt_itk.GetSpacing())
resample.SetSize(gt_itk.GetSize())
resample.SetOutputDirection(gt_itk.GetDirection())
resample.SetOutputOrigin(gt_itk.GetOrigin())
resample.SetTransform(sitk.Transform())
resample.SetDefaultPixelValue(gt_itk.GetPixelIDValue())
resample.SetInterpolator(sitk.sitkNearestNeighbor)
new_pred_itk = resample.Execute(pred_itk)

gt_array = sitk.GetArrayViewFromImage(gt_itk)
pred_array = sitk.GetArrayViewFromImage(new_pred_itk)





        # pdb.set_trace()
        # analysis = AnalyzeVolume(gt_array, pred_array, fixed_img_name='MR', moving_img_name='PCCT')
        # scores = analysis.evaluation_metrics()
        # print(scores)
        # pdb.set_trace()
        # analysis.create_scatter_plot(scores)
    # else:
    #     continue


In [14]:
print('pred_array:', pred_array.min())
print('gt_array:', gt_array.min())


pred_array: 0
gt_array: 0


In [16]:
# pdb.set_trace()

overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
overlap_measures_filter.Execute(gt_itk, new_pred_itk)
dsc = overlap_measures_filter.GetDiceCoefficient()
# hausdorff_distance_filter = sitk.HausdorffDistanceImageFilter()
# hausdorff_distance_filter.Execute(gt_itk, new_pred_itk)
# hdd = hausdorff_distance_filter.GetHausdorffDistance()

print('Dice: \t', dsc)# '\n Hausdorff disctance: \t', hdd)

Dice: 	 0.0


In [18]:
from utils import Image_Registration
import os
import pdb
# pdb.set_trace()
fixed_image = r"E:07_2017_MR.nrrd"
moving_image = r"E:\ME_data_mhd\07_2017\07_2017.mhd"

# root_MR_dir = r"E:\MR_masks_shrunk" #r"E:\TUe_masked_MR"
# root_PCCT_dir =  r"E:\segmentations PCCT Tim\bone_segs\nrrd"

results_path = r"E:\TEST_full_img"

elastix_path = os.path.join(r'C:\Tim\Software\Elastix\elastix.exe')
transformix_path = os.path.join(r'C:\Tim\Software\Elastix\transformix.exe')
affine_parameter_file = r'C:\Users\T2025\Desktop\cadaver_knee_study\code\elastix_parameter_files\parameters_MR_pcct.txt'
bspline_parameter_file = r'C:\Users\T2025\Desktop\cadaver_knee_study\code\elastix_parameter_files\Parameters_BSpline_adjusted.txt'


a = Image_Registration()

a.initialize_elastix(elastix_path, transformix_path, 
                         parameter_files=[affine_parameter_file, bspline_parameter_file], 
                         fixed_image=fixed_image, moving_image=moving_image, 
                         fixed_im_mask=None, results_path=results_path,
                        #  anatomical_structure='femur'
                         )
a.run_elastix()



segmentation_file = r'E:\TUe_segmentations_PCCT\07_2017\07_2017_femoral_cartilage.nrrd' # PCCT

# pdb.set_trace()
transformation_file_dir = os.path.join(results_path, '07_2017')
a.apply_transformation(transformation_file_dir, segmentation_file, transformix_path=transformix_path)

C:\Tim\Software\Elastix\elastix.exe -f E:07_2017_MR.nrrd -m E:\ME_data_mhd\07_2017\07_2017.mhd -p C:\Users\T2025\Desktop\cadaver_knee_study\code\elastix_parameter_files\parameters_MR_pcct.txt -p C:\Users\T2025\Desktop\cadaver_knee_study\code\elastix_parameter_files\Parameters_BSpline_adjusted.txt -out E:\TEST_full_img\07_2017


TransformixError: Transformix crashed with code 1 for command 'C:\Tim\Software\Elastix\transformix.exe -tp E:\TEST_full_img\TransformParameters.0.txt -out E:\TEST_full_img\transformix_results0 -in E:\TUe_segmentations_PCCT\07_2017\07_2017_femoral_cartilage.nrrd'.